In [ ]:
import pandas as pd
import datetime as dt
pd.options.mode.chained_assignment = None

In [ ]:
trades = pd.read_csv("bitpanda-trades.csv", sep=";")

In [ ]:
trades['Timestamp2'] = pd.to_datetime(trades['Timestamp2'])
trades["year"] =  trades['Timestamp2'].dt.year
trades[["Asset market price"]] = trades[["Asset market price"]].apply(pd.to_numeric, errors="coerce")
trades[["Amount Asset"]] = trades[["Amount Asset"]].apply(pd.to_numeric, errors="coerce")

In [ ]:
years = trades["year"].unique()
assets = list(trades["Asset"].unique())

In [ ]:
def get_empty_yearly_profits():
    yearly_profits = {}
    for year in years:
        profit_year = {"untaxable_profit": 0.0, "taxable_profit":0.0,
                       "total_profit": 0.0, "profit_per_asset":{}}
        for asset in assets:
            profit_year["profit_per_asset"][asset] = {}
            profit_year["profit_per_asset"][asset]["taxable_profit"] = 0.0
            profit_year["profit_per_asset"][asset]["untaxable_profit"] = 0.0
            profit_year["profit_per_asset"][asset]["total_profit"] = 0.0
        yearly_profits[year] = profit_year
    return yearly_profits

## Buy / sell tables



In [ ]:
def get_buy_sell_tables():
    buy_tables = {}
    sell_tables = {}
    for asset in assets:
        buy_tables[asset] = trades[(trades["Asset"] == asset) & (trades["Transaction Type"] == "buy")].copy()
        buy_tables[asset]["leftovers"] = buy_tables[asset]["Amount Asset"]
        buy_tables[asset] = buy_tables[asset].reset_index(drop=True)
        sell_tables[asset] = trades[(trades["Asset"] == asset) & (trades["Transaction Type"] == "sell")].copy()
        sell_tables[asset]["leftovers"] = sell_tables[asset]["Amount Asset"]
        sell_tables[asset] = sell_tables[asset].reset_index(drop=True)

    return buy_tables, sell_tables
buy_tables, sell_tables = create_buy_sell_tables()

In [ ]:
iter_show = 6
asset = "DASH"
col_selection = ["Timestamp2", "Transaction Type", "Amount Fiat", "Amount Asset", "Asset", "Asset market price",
                "year", "leftovers"]
buy_tables[asset][:][col_selection]

In [ ]:
sell_tables[asset][:][col_selection]

In [ ]:
#assets = ["DASH"]
yearly_profits = get_empty_yearly_profits()
buy_tables, sell_tables = get_buy_sell_tables()
for asset in assets:
    buy_iter = 0
    sell_iter = 0
    while sell_iter  < len(sell_tables[asset]) and buy_iter < len(buy_tables[asset]):
        subtraction_amount = min(sell_tables[asset]["leftovers"][sell_iter], buy_tables[asset]["leftovers"][buy_iter])

        profits = (sell_tables[asset]["Asset market price"][sell_iter] * subtraction_amount) -  \
                    (buy_tables[asset]["Asset market price"][buy_iter] * subtraction_amount)

        days_diff = (sell_tables[asset]["Timestamp2"][sell_iter] - buy_tables[asset]["Timestamp2"][buy_iter]).days
        if days_diff > 365:
            yearly_profits[sell_tables[asset]["year"][sell_iter]]["profit_per_asset"][asset]["untaxable_profit"] += profits
            yearly_profits[sell_tables[asset]["year"][sell_iter]]["untaxable_profit"] += profits
        else:
            yearly_profits[sell_tables[asset]["year"][sell_iter]]["profit_per_asset"][asset]["taxable_profit"] += profits
            yearly_profits[sell_tables[asset]["year"][sell_iter]]["taxable_profit"] += profits
        yearly_profits[sell_tables[asset]["year"][sell_iter]]["profit_per_asset"][asset]["total_profit"] += profits
        yearly_profits[sell_tables[asset]["year"][sell_iter]]["total_profit"] += profits

        leftovers = sell_tables[asset]["leftovers"][sell_iter] - buy_tables[asset]["leftovers"][buy_iter]

        print(f"Adding profits: {profits}; sell_iter: {sell_iter}; buy_iter: {buy_iter} " + \
             f"for asset: {asset}; days_diff:{days_diff}; leftovers: {leftovers}.")
        if leftovers < 0:
            buy_tables[asset]["leftovers"][buy_iter] = buy_tables[asset]["leftovers"][buy_iter] - subtraction_amount
            sell_tables[asset]["leftovers"][sell_iter] = 0.0
            sell_iter += 1
        else:
            buy_tables[asset]["leftovers"][buy_iter] = 0.0
            sell_tables[asset]["leftovers"][sell_iter] = sell_tables[asset]["leftovers"][sell_iter] - subtraction_amount
            buy_iter += 1

In [ ]:
yearly_profits